# CNN-FaceNet-Face-Recognition

Welcome to a new CNN project!

In this project, we are going to use pre-trained model [FaceNet](https://arxiv.org/pdf/1503.03832.pdf) to solve two common face recognition problems:
- **Face Verification** "Is this the claimed person?" For example, at some airports, you can pass through customs by letting a system scan your passport and then verifying that you (the person carrying the passport) are the correct person. A mobile phone that unlocks using your face is also using face verification. This is a 1:1 matching problem.

- **Face Recognition** "Who is this person?" For example, the video lecture showed a [face recognition video](https://www.youtube.com/watch?v=wr4rx0Spihs) of Baidu employees entering the office without needing to otherwise identify themselves. This is a 1:K matching problem.



## Table of Contents
- [1 - Set up the working directory & Import packages ](#1)
- [2 - Load pre-trained model ](#2)
- [3 - Build the database of the encoded images](#3)
- [4 - Apply the model for Face Verification](#4)
- [5 - Apply the model for Face Regconition](#5)


<a name='1'></a>
## 1 - Set up the working directory & Import packages ##

In [3]:
# Move to the working directory on Google Drive as using Google Colab
import os

if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  PROJECT_ROOT ="/content/drive/MyDrive/GitHub/CNN-FaceNet-Face-Recognition"
  os.chdir(PROJECT_ROOT)
  !pwd
else:
  PROJECT_ROOT ="."

Running on CoLab
/content/drive/MyDrive/GitHub/CNN-FaceNet-Face-Recognition


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# View Nvidia CUDA drivers
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [6]:
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
import PIL

%matplotlib inline
%load_ext autoreload
%autoreload 2


<a name='2'></a>
## 2 - Load Pre-trained model to Encode a Face Image into a 128-Dimensional Vector##


In [7]:
# Load the model
json_file = open(PROJECT_ROOT + '/keras-facenet-h5/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights(PROJECT_ROOT + '/keras-facenet-h5/model.h5')

In [8]:
# Map each person's name to a 128-dimensional encoding of their face.
def img_to_encoding(image_path, model):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(160, 160))
    img = np.around(np.array(img) / 255.0, decimals=12)
    x_train = np.expand_dims(img, axis=0)
    embedding = model.predict_on_batch(x_train)
    return embedding / np.linalg.norm(embedding, ord=2)

<a name='3'></a>
## 3 - Build the database of the encoded images ##


In [11]:
# build the database
database = {}
database["younes"] = img_to_encoding(PROJECT_ROOT + "/images_orig/younes.jpg", model)
database["tian"] = img_to_encoding(PROJECT_ROOT + "/images_orig/tian.jpg", model)
database["andrew"] = img_to_encoding(PROJECT_ROOT + "/images_orig/andrew.jpg", model)
database["kian"] = img_to_encoding(PROJECT_ROOT + "/images_orig/kian.jpg", model)
database["dan"] = img_to_encoding(PROJECT_ROOT + "/images_orig/dan.jpg", model)
database["sebastiano"] = img_to_encoding(PROJECT_ROOT + "/images_orig/sebastiano.jpg", model)
database["bertrand"] = img_to_encoding(PROJECT_ROOT + "/images_orig/bertrand.jpg", model)
database["kevin"] = img_to_encoding(PROJECT_ROOT + "/images_orig/kevin.jpg", model)
database["felix"] = img_to_encoding(PROJECT_ROOT + "/images_orig/felix.jpg", model)
database["benoit"] = img_to_encoding(PROJECT_ROOT + "/images_orig/benoit.jpg", model)
database["arnaud"] = img_to_encoding(PROJECT_ROOT + "/images_orig/arnaud.jpg", model)

<a name='4'></a>
## 4 - Apply the model for Face Verification ##

 Build a function that verifies if the person on the "image_path" image is "identity".

In [12]:
def verify(image_path, identity, database, model,dist_thres=1.0):
    """
    Function that verifies if the person on the "image_path" image is "identity".
    
    Arguments:
        image_path -- path to an image
        identity -- string, name of the person you'd like to verify the identity. Has to be an employee who works in the office.
        database -- python dictionary mapping names of allowed people's names (strings) to their encodings (vectors).
        model -- your Inception model instance in Keras
    
    Returns:
        dist -- distance between the image_path and the image of "identity" in the database.
        door_open -- True, if the door should open. False otherwise.
    """
    ### START CODE HERE
    # Step 1: Compute the encoding for the image. Use img_to_encoding() see example above. (≈ 1 line)
    encoding = img_to_encoding(image_path, model)
    # Step 2: Compute distance with identity's image (≈ 1 line)
    dist = np.linalg.norm(encoding-database[identity])
    # Step 3: Open the door if dist < 1.0, else don't open (≈ 3 lines)
    if dist < dist_thres:
        print("Correct! It's " + str(identity))
        door_open = True
    else:
        print("Incorrect! It's not " + str(identity))
        door_open = False
    ### END CODE HERE        
    return dist, door_open

Testing

In [13]:
recorded_person = PROJECT_ROOT + "/images_tested/camera_0.jpg"
verify(recorded_person, "younes", database, model)

Correct! It's younes


(0.59929454, True)

<a name='5'></a>
## 5 - Apply the model for Face Regconition##

 Build a function that finds who is the person on the image_path image.

In [14]:
def who_is_it(image_path, database, model,dist_thres=1.0):
    """
    Implements face recognition for the office by finding who is the person on the image_path image.
    
    Arguments:
        image_path -- path to an image
        database -- database containing image encodings along with the name of the person on the image
        model -- your Inception model instance in Keras
    
    Returns:
        min_dist -- the minimum distance between image_path encoding and the encodings from the database
        identity -- string, the name prediction for the person on image_path
    """
    
    ### START CODE HERE

    ## Step 1: Compute the target "encoding" for the image. Use img_to_encoding() see example above. ## (≈ 1 line)
    encoding =  img_to_encoding(image_path, model)
    
    ## Step 2: Find the closest encoding ##
    
    # Initialize "min_dist" to a large value, say 100 (≈1 line)
    min_dist = 100
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        
        # Compute L2 distance between the target "encoding" and the current db_enc from the database. (≈ 1 line)
        dist = np.linalg.norm(encoding-db_enc)

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
        if dist<min_dist:
            min_dist = dist
            identity = name
    ### END CODE HERE
    
    if min_dist > dist_thres:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

 Let's see if our who_it_is() algorithm identifies a person in the database.

In [15]:
# Test 1 with Younes pictures 
recorded_person = PROJECT_ROOT + "/images_tested/camera_0.jpg"
who_is_it(recorded_person, database, model)

it's younes, the distance is 0.59929454


(0.59929454, 'younes')

In [16]:
image_path = 'images_orig/younes.jpg'
encoding = img_to_encoding(image_path, model)
encoding

array([[-0.04321312,  0.02769636, -0.11006935, -0.02181004,  0.0361076 ,
        -0.0456748 ,  0.01104234,  0.01584502, -0.06034191, -0.06126551,
         0.03670533, -0.0961697 ,  0.10318674, -0.0190394 ,  0.12444156,
         0.14966859, -0.00199561, -0.15202133, -0.08695692,  0.1351084 ,
         0.09610432,  0.01601146, -0.04747052,  0.09019998, -0.00961677,
        -0.05942307,  0.02899263, -0.02146621, -0.15220144,  0.03687984,
         0.04161033, -0.03324274,  0.03212211,  0.15484709, -0.03253124,
         0.13734728, -0.03606853, -0.19555251, -0.03386902,  0.138185  ,
         0.01098225,  0.05590543,  0.07494756,  0.1048388 ,  0.04818638,
        -0.08864869,  0.18727283,  0.09609108,  0.07169706,  0.00777906,
        -0.1526765 ,  0.02436365, -0.13910365,  0.00391132, -0.00524655,
        -0.14967903, -0.12527572,  0.1339501 ,  0.12619701,  0.12007786,
         0.00050071, -0.0457189 , -0.02791403, -0.07097332,  0.0454842 ,
        -0.08937535, -0.10754488,  0.07807683, -0.0

In [18]:
img = tf.keras.preprocessing.image.load_img(image_path, target_size=(160, 160))
img = np.around(np.array(img) / 255.0, decimals=12)
img

array([[[1.        , 1.        , 0.70588235],
        [1.        , 1.        , 0.70588235],
        [0.95686275, 0.94901961, 0.76078431],
        ...,
        [0.74901961, 0.82352941, 0.56470588],
        [0.54117647, 0.62352941, 0.36862745],
        [0.54117647, 0.62352941, 0.36862745]],

       [[1.        , 1.        , 0.70588235],
        [1.        , 1.        , 0.70588235],
        [0.95686275, 0.94901961, 0.76078431],
        ...,
        [0.74901961, 0.82352941, 0.56470588],
        [0.54117647, 0.62352941, 0.36862745],
        [0.54117647, 0.62352941, 0.36862745]],

       [[0.98431373, 0.97647059, 0.7372549 ],
        [0.98431373, 0.97647059, 0.7372549 ],
        [1.        , 1.        , 0.84313725],
        ...,
        [0.65882353, 0.70196078, 0.46666667],
        [0.69411765, 0.74901961, 0.50980392],
        [0.69411765, 0.74901961, 0.50980392]],

       ...,

       [[0.99607843, 0.99607843, 0.98823529],
        [0.99607843, 0.99607843, 0.98823529],
        [0.99607843, 0